In [ ]:
pip install wandb

In [ ]:
import os
import requests
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoImageProcessor, AutoModel, AutoModelForImageClassification, Trainer, TrainingArguments
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score
from io import BytesIO

In [ ]:
urltest = "https://huggingface.co/datasets/Falah/Alzheimer_MRI/resolve/main/data/test-00000-of-00001-44110b9df98c5585.parquet"
urltrain = "https://huggingface.co/datasets/Falah/Alzheimer_MRI/resolve/main/data/train-00000-of-00001-c08a401c53fe5312.parquet"
output_file_test = "test_data.parquet"
output_file_train = "train_data.parquet"

if not os.path.exists(output_file_train):
    print("Downloading the dataset")
    response = requests.get(urltrain)
    with open(output_file_train, 'wb') as f:
        f.write(response.content)
    print("Download complete")
else:
    print("File already exists")

if not os.path.exists(output_file_test):
    print("Downloading the dataset")
    response = requests.get(urltest)
    with open(output_file_test, 'wb') as f:
        f.write(response.content)
    print("Download complete")
else:
    print("File already exists")

data_train = pd.read_parquet(output_file_train)
data_test = pd.read_parquet(output_file_test)

print(data_train.head())

File already exists.
File already exists.
Loading the dataset...
Dataset loaded.
                                               image  label
0  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...      2
1  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...      0
2  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...      3
3  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...      3
4  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...      2


In [ ]:
class AlzheimerDataset(Dataset):
    def __init__(self, dataframe, processor):
        self.data = dataframe
        self.processor = processor
        self.mode = mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_data = self.data.iloc[idx]['image']['bytes']
        label = self.data.iloc[idx]['label']
        image = Image.open(BytesIO(image_data)).convert("RGB")
        encoding = self.processor(images=image, return_tensors="pt")
        return {
            "pixel_values": encoding["pixel_values"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

processor = AutoImageProcessor.from_pretrained("Falah/Alzheimer_classification_model")

train_dataset = AlzheimerDataset(data_train, processor)
test_dataset = AlzheimerDataset(data_test, processor)

model = AutoModelForImageClassification.from_pretrained(
    "Falah/Alzheimer_classification_model", num_labels=4
)

for param in model.base_model.parameters():
    param.requires_grad = False

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

os.environ["WANDB_DISABLED"] = "true"

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor,
    compute_metrics=compute_metrics
)

trainer.train()

print("Evaluating the model on test data")
metrics = trainer.evaluate(test_dataset)
print("Test Metrics:", metrics)

trainer.save_model("./fine_tuned_model")
processor.save_pretrained("./fine_tuned_processor")


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-9-7882eb48ec26>:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.124900,0.274150,0.898438,0.896106


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.124900,0.274150,0.898438,0.896106
2,0.089300,0.271770,0.904687,0.904023
3,0.083200,0.271520,0.903906,0.903483
4,0.122000,0.271190,0.903125,0.902883
5,0.103700,0.271311,0.904687,0.904588


Evaluating the model on test data...


Test Metrics: {'eval_loss': 0.27177000045776367, 'eval_accuracy': 0.9046875, 'eval_f1': 0.9040234403890601, 'eval_runtime': 890.4811, 'eval_samples_per_second': 1.437, 'eval_steps_per_second': 0.18, 'epoch': 5.0}


['./fine_tuned_processor/preprocessor_config.json']